In [1]:
import sys
import datetime
sys.path.append("../") # go to parent dir
from util.read_data import DataReader
from util.evaluator import ModelEvaluator

In this notebook, we will usually use normalized features and volume. The volume will be denormalized only for the purpose of model evaluation. We will use StandardScaler for normalization.

We can draw conclusion from the Exploratory Data Analysis that not all features from the original dataset are usefull for prediction and few more additional features can be added. Models in this notebook will work with usually with following columns:

| Feature  | Type | Description |
| ------------- | ------------- ||
| Volume X  | float64  |Historial trading volume shifted by X|
| AdjCloseDiff X  | float64  |Historical difference between AdjClose price of two consecutive days shifted by X|
| HighLowDiff X  | float64  ||Historical difference between High and Low price shifted by X|
| DayOfWeek X  | one-hot |One-hot encoding value for each day|
| Month X  | one-hot  |One-hot encoding value for each month|

Le'ts now read the normalized data.

In [2]:
reader = DataReader()
df = reader.read_all_data_normalized()
evaluator = ModelEvaluator(reader.label_scaler)

df_test_features, series_test_volume = reader.get_test_data(df)

Baseline model is very simple, predicting the same value of Volume as yesterday: $\hat{E}[v_{d+1}]=v_d$

In [3]:
def baseline_model(volume):
    pred_volume = volume.shift(1)
    return pred_volume

In [4]:
baseline_pred_volume = baseline_model(series_test_volume)
# the first value is nan, we will ignore them by mapping [1:]
baseline_model_results = evaluator.evaluate("baseline", series_test_volume[1:], baseline_pred_volume[1:])
print(baseline_model_results)

baseline: MSE = 3.684400e+17, R2 = 0.033, conf. int. 95% of error = (-53,069,427 - 53,341,067)


The baseline r2 score is **0.033**, which is slightly better than predicting average value.

Another simple model would use moving average over some time window.
The model is following: $V_{d+1}=\frac 1n \sum_{i=0}^{n-1} V_{d-i}$.
We can try multiple n values, le'ts try following values: {1, 3, 5, 10, 50, 100}

In [5]:
def moving_average_model(volume, n):
    pred_volume = volume.rolling(n).mean().shift(1)
    return pred_volume

In [6]:
window_size = [1,2,3,4,5,8,10,15,20,50]
# first few initial values will have nan values, we will ignorem them by mapping [n:]
moving_average_result_list = [evaluator.evaluate("moving_average {}".format(n), 
    series_test_volume[n:],
    moving_average_model(series_test_volume, n)[n:]) 
    for n in window_size]

for result in moving_average_result_list:
    print(result)


moving_average 1: MSE = 3.684400e+17, R2 = 0.033, conf. int. 95% of error = (-53,069,427 - 53,341,067)
moving_average 2: MSE = 3.136891e+17, R2 = 0.178, conf. int. 95% of error = (-49,344,246 - 48,940,358)
moving_average 3: MSE = 3.099159e+17, R2 = 0.189, conf. int. 95% of error = (-49,530,282 - 48,259,465)
moving_average 4: MSE = 3.130410e+17, R2 = 0.182, conf. int. 95% of error = (-50,315,382 - 48,064,990)
moving_average 5: MSE = 3.128615e+17, R2 = 0.184, conf. int. 95% of error = (-52,614,748 - 45,836,522)
moving_average 8: MSE = 3.332384e+17, R2 = 0.136, conf. int. 95% of error = (-59,016,798 - 42,898,679)
moving_average 10: MSE = 3.298599e+17, R2 = 0.147, conf. int. 95% of error = (-62,279,075 - 39,324,762)
moving_average 15: MSE = 3.368760e+17, R2 = 0.136, conf. int. 95% of error = (-68,739,344 - 34,466,184)
moving_average 20: MSE = 3.577937e+17, R2 = 0.089, conf. int. 95% of error = (-73,827,894 - 33,084,952)
moving_average 50: MSE = 3.892483e+17, R2 = 0.057, conf. int. 95% of e

The moving average models improve the baseline , with the best R2 score = **0.189** for window size = 3. The window size 1 also confirms that the algorithm has correct implementation, as it has the same results as the baseline model.

Le'ts try to use exponential moving average.

In [7]:
def exp_moving_average_model(volume, n):
    pred_volume = volume.ewm(span=n).mean().shift(1)
    return pred_volume

In [8]:
window_size = [1,2,3,4,5,8,10,15,20,50]
exp_moving_average_result_list = [evaluator.evaluate("moving_average {}".format(n), 
        series_test_volume[n:],
        exp_moving_average_model(series_test_volume, n)[n:])
                                                 for n in window_size]

for result in exp_moving_average_result_list:
    print(result)


moving_average 1: MSE = 3.684400e+17, R2 = 0.033, conf. int. 95% of error = (-53,069,427 - 53,341,067)
moving_average 2: MSE = 3.024401e+17, R2 = 0.207, conf. int. 95% of error = (-48,491,694 - 48,014,561)
moving_average 3: MSE = 2.880625e+17, R2 = 0.246, conf. int. 95% of error = (-48,026,359 - 46,252,592)
moving_average 4: MSE = 2.844844e+17, R2 = 0.257, conf. int. 95% of error = (-49,530,117 - 44,255,685)
moving_average 5: MSE = 2.845396e+17, R2 = 0.258, conf. int. 95% of error = (-51,352,553 - 42,536,855)
moving_average 8: MSE = 2.907816e+17, R2 = 0.246, conf. int. 95% of error = (-54,959,423 - 40,242,576)
moving_average 10: MSE = 2.953661e+17, R2 = 0.237, conf. int. 95% of error = (-58,118,071 - 38,026,698)
moving_average 15: MSE = 3.063394e+17, R2 = 0.215, conf. int. 95% of error = (-64,181,541 - 34,235,284)
moving_average 20: MSE = 3.151585e+17, R2 = 0.198, conf. int. 95% of error = (-67,282,532 - 33,058,379)
moving_average 50: MSE = 3.594106e+17, R2 = 0.129, conf. int. 95% of e

Exponential moving average again improves the score, with the best R2 score = **0.258** for window size = 5.

Let's now try to learn linear regression, on the normalized historical features described in the table on the top of the notebook. The time window size will be variable. 

In [14]:
from sklearn.linear_model import LinearRegression

def linear_regression_model(features, volume):
    regressor = LinearRegression()
    
    regressor.fit(features, volume)
    return regressor

def eval_model(name, model_function, n, custom_callback=None, train_date_from = None):
    # prepare the features, first rows with nan values will not be included
    df_enriched = reader.prepare_window_features_for_training(df, n)[n:]
    if (train_date_from != None):
        df_enriched = df_enriched.loc[train_date_from:]

    train_features, train_volume = reader.get_train_data(df_enriched)
    test_features, test_volume = reader.get_test_data(df_enriched)

    regressor = model_function(train_features, train_volume)
    
    # allows to custom-enrich the results output
    if (custom_callback):
        random_forest_callback(train_features, regressor)
    
    print(evaluator.evaluate("{} {} on train".format(name, n), 
                             regressor.predict(train_features), train_volume))
    result = evaluator.evaluate("{} {} on test ".format(name, n), 
                                regressor.predict(test_features), test_volume)
    print(result)
    print()
    return result


In [15]:
for n in (1,3,5,10,50,100):
    eval_model("lin. reg.", linear_regression_model, n)

lin. reg. 1 on train: MSE = 3.346656e+17, R2 = 0.842, conf. int. 95% of error = (-17,339,747 - 17,339,747)
lin. reg. 1 on test : MSE = 3.178947e+17, R2 = -0.090, conf. int. 95% of error = (3,956,311 - 102,699,885)

lin. reg. 3 on train: MSE = 2.906242e+17, R2 = 0.866, conf. int. 95% of error = (-16,162,359 - 16,162,359)
lin. reg. 3 on test : MSE = 2.635186e+17, R2 = -0.092, conf. int. 95% of error = (-6,622,419 - 83,280,301)

lin. reg. 5 on train: MSE = 2.809084e+17, R2 = 0.871, conf. int. 95% of error = (-15,853,735 - 15,933,504)
lin. reg. 5 on test : MSE = 2.545230e+17, R2 = -0.114, conf. int. 95% of error = (-7,600,602 - 80,754,319)

lin. reg. 10 on train: MSE = 2.721709e+17, R2 = 0.875, conf. int. 95% of error = (-15,669,396 - 15,637,903)
lin. reg. 10 on test : MSE = 2.511179e+17, R2 = -0.187, conf. int. 95% of error = (-11,347,578 - 76,414,330)

lin. reg. 50 on train: MSE = 2.548655e+17, R2 = 0.883, conf. int. 95% of error = (-15,216,464 - 15,222,191)
lin. reg. 50 on test : MSE = 

Linera regression does not seem to have very good results. The best R2 score on the test set is still not as good as predicting mean value.

Let's not try to use another, random forest regressor, with the same features.

In [29]:
from sklearn.ensemble import RandomForestRegressor

def random_forest_model(features, volume):
    regressor = RandomForestRegressor(n_estimators=100)
    
    regressor.fit(features, volume)
    return regressor

def random_forest_callback(train_features, regressor):
    importances = list(zip(train_features.columns, regressor.feature_importances_))
    importances.sort(key=lambda x: x[1], reverse = True)
    print("First 5 Feature importances:".format(n))
    for importance in importances[:5]:
        print('Feature: {:30} \t Importance: {}'.format(importance[0], importance[1]))


In [28]:
for n in (1,3,5,10,20):
    eval_model("random_forest", random_forest_model, n, random_forest_callback)

First 5 Feature importances:
Feature: Volume1                        	 Importance: 0.8946135869153913
Feature: HighLowDiff1                   	 Importance: 0.03070479293932718
Feature: AdjCloseDiff1                  	 Importance: 0.028191001466756384
Feature: DayOfWeek_Monday               	 Importance: 0.00903923260466571
Feature: Month_12                       	 Importance: 0.0043292836188353855
random_forest 1 on train: MSE = 4.762153e+16, R2 = 0.979, conf. int. 95% of error = (-7,766,657 - 5,315,185)
random_forest 1 on test : MSE = 3.048690e+17, R2 = 0.002, conf. int. 95% of error = (-108,384,749 - -11,685,343)

First 5 Feature importances:
Feature: Volume1                        	 Importance: 0.8294453893401988
Feature: Volume2                        	 Importance: 0.06120432738544723
Feature: Volume3                        	 Importance: 0.027123367870980578
Feature: AdjCloseDiff1                  	 Importance: 0.012691976813380738
Feature: HighLowDiff1                   	 Importan

It is obvious from the comparison between train and test results that the model has high variance, it overfits to the train set. The best R2 score is around **0.010** (changes with each run), for window size = 5  The decision tree regression model is not suitable for the task.

**Insights:**
* Model easily overfits to the train set
* The sorted list of feature importances shows that the most important feature for decission is Volume(t-1), which is expected result.  
* The AdjCloseDiff1(t-1) and HighLowDiff1(t-1) does not seem important at all


is Le't see SVM model, for comparison.

In [30]:
from sklearn.svm import SVR

def svm_model(features, volume):
    regressor = SVR(gamma='auto')
    
    regressor.fit(features, volume)
    return regressor


In [33]:
for n in (1, 2, 3,5,10,20):
    eval_model("svm_model", svm_model, n)

svm_model 1 on train: MSE = 2.952591e+17, R2 = 0.859, conf. int. 95% of error = (23,015,390 - 55,589,224)
svm_model 1 on test : MSE = 3.036388e+17, R2 = 0.069, conf. int. 95% of error = (-7,893,085 - 88,611,036)

svm_model 2 on train: MSE = 2.541464e+17, R2 = 0.879, conf. int. 95% of error = (17,333,138 - 47,557,698)
svm_model 2 on test : MSE = 2.722919e+17, R2 = -0.000, conf. int. 95% of error = (1,910,299 - 93,297,325)

svm_model 3 on train: MSE = 2.413306e+17, R2 = 0.886, conf. int. 95% of error = (15,724,379 - 45,180,461)
svm_model 3 on test : MSE = 2.640269e+17, R2 = 0.018, conf. int. 95% of error = (-7,293,575 - 82,695,807)

svm_model 5 on train: MSE = 2.226577e+17, R2 = 0.894, conf. int. 95% of error = (11,552,309 - 39,852,495)
svm_model 5 on test : MSE = 2.735825e+17, R2 = -0.020, conf. int. 95% of error = (-33,548,592 - 58,054,761)

svm_model 10 on train: MSE = 2.003799e+17, R2 = 0.904, conf. int. 95% of error = (12,700,390 - 39,563,225)
svm_model 10 on test : MSE = 2.803728e+

The best SVM model is again overfitted to the train set. The best R2 score is around 0.069 (changing with each run). 

It seem that we are not moving forward a lot with traditional ML models. Let's try one last model,  ridge regression with cross validation.

In [36]:
from sklearn import linear_model

def linear_ridge_model(features, volume):
    regressor = linear_model.RidgeCV(alphas=[0.1, 0.5, 1.0, 5, 10.0], cv=5)
    
    regressor.fit(features, volume)
    return regressor

In [38]:
for n in (1, 2, 3,5,10,20):
    eval_model("linear_ridge_model", linear_ridge_model, n)

/home/martin/work/github/trade_volume_prediction/venv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


linear_ridge_model 1 on train: MSE = 3.346656e+17, R2 = 0.842, conf. int. 95% of error = (-17,339,747 - 17,339,747)
linear_ridge_model 1 on test : MSE = 3.178889e+17, R2 = -0.090, conf. int. 95% of error = (3,962,585 - 102,705,255)

linear_ridge_model 2 on train: MSE = 2.970075e+17, R2 = 0.862, conf. int. 95% of error = (-16,336,977 - 16,336,977)
linear_ridge_model 2 on test : MSE = 2.728444e+17, R2 = -0.079, conf. int. 95% of error = (-4,418,830 - 87,060,873)

linear_ridge_model 3 on train: MSE = 2.906243e+17, R2 = 0.866, conf. int. 95% of error = (-16,162,359 - 16,162,359)
linear_ridge_model 3 on test : MSE = 2.635157e+17, R2 = -0.092, conf. int. 95% of error = (-6,620,907 - 83,281,329)

linear_ridge_model 5 on train: MSE = 2.809087e+17, R2 = 0.871, conf. int. 95% of error = (-15,893,628 - 15,893,628)
linear_ridge_model 5 on test : MSE = 2.545127e+17, R2 = -0.114, conf. int. 95% of error = (-7,640,096 - 80,713,027)

linear_ridge_model 10 on train: MSE = 2.721949e+17, R2 = 0.875, conf

The best R2 score with RidgeCV model on the test set is worse than predicting average Volume.

The best model so far was simple model with exponential moving averate prediction.
The next possible step with current data we have could be using exponential average with various spans as the features for linear models.

**Hypothesis** More rewarding would probably be using more informative features. One of the possible such features could be number of expected earning announcements, ideally weighted by the market value of the company.

**Hypothesis** No model with the features used does not have very good performance. The hypothesis is that **historical Market data on its own are not very good features to predict next trade Volume. The market reaches equilibrium at the end of each date and all available information was already processed.**
